In [0]:
#connecting via Service Principle (using secret scope)
spark.conf.set("fs.azure.account.auth.type.adlsg2datalake.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adlsg2datalake.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adlsg2datalake.dfs.core.windows.net", "37a372de-1804-4a80-b98f-54ba7ef17ef3")
spark.conf.set("fs.azure.account.oauth2.client.secret.adlsg2datalake.dfs.core.windows.net", dbutils.secrets.get(scope="databricks_secret_scope_soham", key="secret-adlsg2-service-principle-value"))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adlsg2datalake.dfs.core.windows.net", "https://login.microsoftonline.com/9adebd2f-da4d-4f91-9e68-91d049a77ef2/oauth2/token")

In [0]:
#display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net"))
display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_sample_data/"))
#display(dbutils.fs.ls("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_cricket_files/"))


In [0]:
df = spark.read.option("multiline", "true").format("json").load("abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/source_json_cricket_files/")


In [0]:
#df.display()
#df.count()
df.createOrReplaceTempView("RAW_JSON")

In [0]:
%sql
--SELECT CAST(parse_json('{"key": 123, "data": 5.1 }') AS MAP<STRING, DECIMAL(20, 1)>);  ---  {"data":"5.1","key":"123.0"}

SELECT
  --INFO.*
  --INFO.PLAYERS,
  --INFO.PLAYERS.Bangladesh,
  --CAST(parse_json(INFO.PLAYERS) AS MAP< STRUCT, STRUCT>  ) AS C1
  --CAST(parse_json(INFO.PLAYERS) AS MAP< STRUCT<STRING:ARRAY<STRING>>, STRUCT<STRING:ARRAY<STRING>> >  ) AS C1
  --CAST(to_json(INFO.PLAYERS) AS MAP< ARRAY<STRING>, ARRAY<STRING> >  ) AS C1,
  --TO_JSON(INFO.PLAYERS),
  
  PLYR_IDX,
  REPLACE(REPLACE(PLYR_VAL,'{',''),'}','')
  --(REPLACE(REPLACE(PLYR_VAL,'{',''),'}',''))
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(SPLIT(TO_JSON(INFO.PLAYERS),'],"')) AS PLYR_IDX, PLYR_VAL
;

In [0]:
%sql
SELECT
  --INFO.*
  INFO.PLAYERS
  --CAST(INFO.PLAYERS AS STRING)
  --SPLIT(CAST(INFO.PLAYERS AS STRING), '],"')
  --PLYR_IDX,
  --PLYR_VAL.*
FROM
  RAW_JSON

In [0]:
%sql
--SELECT CAST(parse_json('{"key": 123, "data": 5.1 }') AS MAP<STRING, DECIMAL(20, 1)>);  ---  {"data":"5.1","key":"123.0"}
-- > SELECT CAST(map(struct('Hello', 'World'), 'Greeting') AS MAP<STRUCT<w1:string, w2:string>, string>);      --{{Hello, World}->Greeting}
SELECT
  --INFO.*
  --INFO.PLAYERS,
  --INFO.PLAYERS.Bangladesh,
  --CAST(parse_json(INFO.PLAYERS) AS MAP< STRUCT, STRUCT>  ) AS C1
  --CAST(parse_json(INFO.PLAYERS) AS MAP< STRUCT<STRING:ARRAY<STRING>>, STRUCT<STRING:ARRAY<STRING>> >  ) AS C1
  --to_json(INFO.PLAYERS),
  --typeof(to_json(INFO.PLAYERS))
  --SPLIT(to_json(INFO.PLAYERS),'],"')
  
  TEAM_IDX,
  --PLYR_VAL,
  SPLIT(TEAM_VAL,':')[0] AS TEAM_NAME,
  --SPLIT(TEAM_VAL,':')[1],
  PLYR_IDX,
  PLYR_VAL AS PLAYER_NAME
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(SPLIT(TO_JSON(INFO.PLAYERS),'],"')) AS TEAM_IDX, TEAM_VAL
LATERAL VIEW POSEXPLODE(SPLIT(SPLIT(TEAM_VAL,':')[1],',')) AS PLYR_IDX, PLYR_VAL
;

In [0]:
%sql
--Method 4:
--Creating External Table in ADLSg2 in Delta format (after creating credentials and external location)


--SELECT CAST(parse_json('{"key": 123, "data": 5.1 }') AS MAP<STRING, DECIMAL(20, 1)>);  ---  {"data":"5.1","key":"123.0"}
-- > SELECT CAST(map(struct('Hello', 'World'), 'Greeting') AS MAP<STRUCT<w1:string, w2:string>, string>);      --{{Hello, World}->Greeting}

CREATE TABLE CRICKET_DB.CRICKET_PLAYERS_MASTER_EXTERNAL
LOCATION 'abfss://databricks-hello-world@adlsg2datalake.dfs.core.windows.net/target_conformed_delta_lakehouse/cricket_db_external_tables/player_master_external'
AS
SELECT
  --INFO.*
  --INFO.PLAYERS,
  TEAM_IDX,
  --PLYR_VAL,
  SPLIT(TEAM_VAL,':')[0] AS TEAM_NAME,
  --SPLIT(TEAM_VAL,':')[1],
  PLYR_IDX,
  PLYR_VAL AS PLAYER_NAME
FROM
  RAW_JSON
LATERAL VIEW POSEXPLODE(SPLIT(TO_JSON(INFO.PLAYERS),'],"')) AS TEAM_IDX, TEAM_VAL
LATERAL VIEW POSEXPLODE(SPLIT(SPLIT(TEAM_VAL,':')[1],',')) AS PLYR_IDX, PLYR_VAL
;